In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from solver import *
from cache import *
from metrics import *
from input_preprocessing import *

In [2]:
device = torch.device('cpu')

x_grid=np.linspace(0,1,21)
t_grid=np.linspace(0,1,21)

x = torch.from_numpy(x_grid)
t = torch.from_numpy(t_grid)

grid = torch.cartesian_prod(x, t).float()

grid.to(device)


tensor([[0.0000, 0.0000],
        [0.0000, 0.0500],
        [0.0000, 0.1000],
        [0.0000, 0.1500],
        [0.0000, 0.2000],
        [0.0000, 0.2500],
        [0.0000, 0.3000],
        [0.0000, 0.3500],
        [0.0000, 0.4000],
        [0.0000, 0.4500],
        [0.0000, 0.5000],
        [0.0000, 0.5500],
        [0.0000, 0.6000],
        [0.0000, 0.6500],
        [0.0000, 0.7000],
        [0.0000, 0.7500],
        [0.0000, 0.8000],
        [0.0000, 0.8500],
        [0.0000, 0.9000],
        [0.0000, 0.9500],
        [0.0000, 1.0000],
        [0.0500, 0.0000],
        [0.0500, 0.0500],
        [0.0500, 0.1000],
        [0.0500, 0.1500],
        [0.0500, 0.2000],
        [0.0500, 0.2500],
        [0.0500, 0.3000],
        [0.0500, 0.3500],
        [0.0500, 0.4000],
        [0.0500, 0.4500],
        [0.0500, 0.5000],
        [0.0500, 0.5500],
        [0.0500, 0.6000],
        [0.0500, 0.6500],
        [0.0500, 0.7000],
        [0.0500, 0.7500],
        [0.0500, 0.8000],
        [0.0

In [3]:
prepared_grid,grid_dict,point_type = grid_prepare(grid)

In [4]:
model = torch.nn.Sequential(
        torch.nn.Linear(2, 100),
        torch.nn.Tanh(),
        torch.nn.Linear(100, 100),
        torch.nn.Tanh(),
        torch.nn.Linear(100, 100),
        torch.nn.Tanh(),
        torch.nn.Linear(100, 2)
    )

In [5]:
def func(grid):
    x, t = grid[:,0],grid[:,1]
    return torch.sin(np.pi * x) * torch.cos(C * np.pi * t) + torch.sin(A * np.pi * x) * torch.cos(
        A * C * np.pi * t
    )
fun = lambda x: 2/np.cosh(x)

A = 2
C = 10
# Initial conditions at t=0
bnd1 = torch.cartesian_prod(x, torch.from_numpy(np.array([0], dtype=np.float64))).float()

# u(0,x)=sin(pi*x)
bndval1_1 = func(bnd1)
bndval1_2 = fun(bnd1[:, 0])
bndval1 = torch.stack((bndval1_1,bndval1_2),dim=1)

# Initial conditions at t=1
bnd2 = torch.cartesian_prod(x, torch.from_numpy(np.array([1], dtype=np.float64))).float()

# u(1,x)=sin(pi*x)
bndval2_1 = func(bnd2)
bndval2_2 = fun(bnd2[:, 0])
bndval2 = torch.stack((bndval2_1,bndval2_2),dim=1)

# Boundary conditions at x=0
bnd3 = torch.cartesian_prod(torch.from_numpy(np.array([0], dtype=np.float64)), t).float()

# u(0,t)=0
bndval3_1 = func(bnd3)
bndval3_2 = fun(bnd3[:, 0])
bndval3 = torch.stack((bndval3_1,bndval3_2),dim=1)

# Boundary conditions at x=1
bnd4 = torch.cartesian_prod(torch.from_numpy(np.array([1], dtype=np.float64)), t).float()

# u(1,t)=0
bndval4_1 = func(bnd4)
bndval4_2 = fun(bnd4[:, 0])
bndval4 = torch.stack((bndval4_1,bndval4_2),dim=1)

# Putting all bconds together
bconds = [[bnd1, bndval1], [bnd2, bndval2], [bnd3, bndval3], [bnd4, bndval4]]

In [ ]:
# Initial conditions at t=0
bnd1 = torch.cartesian_prod(x, torch.from_numpy(np.array([0], dtype=np.float64))).float()

# u(0,x)=sin(pi*x)
bndval1 = func(bnd1)

# Initial conditions at t=1
bnd2 = torch.cartesian_prod(x, torch.from_numpy(np.array([1], dtype=np.float64))).float()

# u(1,x)=sin(pi*x)
bndval2 = func(bnd2)

# Boundary conditions at x=0
bnd3 = torch.cartesian_prod(torch.from_numpy(np.array([0], dtype=np.float64)), t).float()

# u(0,t)=0
bndval3 = func(bnd3)

# Boundary conditions at x=1
bnd4 = torch.cartesian_prod(torch.from_numpy(np.array([1], dtype=np.float64)), t).float()

# u(1,t)=0
bndval4 = func(bnd4)

# Putting all bconds together
bconds = [[bnd1, bndval1], [bnd2, bndval2], [bnd3, bndval3], [bnd4, bndval4]]

In [6]:
schrodinger_eq_real = {
    'du/dt':
        {
            'const': 1,
            'term': [0],
            'power': 1,
            'var': 0
        },
    '1/2*d2v/dx2':
        {
            'const': 1 / 2,
            'term': [1, 1],
            'power': 1,
            'var': 1
        },
    # 'v * u**2':
    #     {
    #         'const': 1,
    #         'term': [[None], [None]],
    #         'power': [1, 2],
    #         'var': [0, 1]
    #     },
    'v**3':
        {
            'const': 1,
            'term': [None],
            'power': 3,
            'var': 1
        }

}

In [ ]:
model(prepared_grid)[0:,0] # нулевой выход
model(prepared_grid)[0:,1] # первый выход

In [7]:
prepared_bconds = bnd_prepare(bconds,prepared_grid,grid_dict,h=0.001)
full_prepared_operator = operator_prepare(schrodinger_eq_real, grid_dict, subset=['central'], true_grid=grid, h=0.001)
prepared_bconds

[[[402,
   422,
   423,
   424,
   425,
   426,
   427,
   428,
   429,
   430,
   431,
   432,
   433,
   434,
   435,
   436,
   437,
   438,
   439,
   440,
   0],
  None,
  tensor([[ 0.0000e+00,  2.0000e+00],
          [ 4.6545e-01,  1.9975e+00],
          [ 8.9680e-01,  1.9900e+00],
          [ 1.2630e+00,  1.9777e+00],
          [ 1.5388e+00,  1.9607e+00],
          [ 1.7071e+00,  1.9391e+00],
          [ 1.7601e+00,  1.9133e+00],
          [ 1.7000e+00,  1.8835e+00],
          [ 1.5388e+00,  1.8500e+00],
          [ 1.2967e+00,  1.8133e+00],
          [ 1.0000e+00,  1.7736e+00],
          [ 6.7867e-01,  1.7314e+00],
          [ 3.6327e-01,  1.6871e+00],
          [ 8.1990e-02,  1.6410e+00],
          [-1.4204e-01,  1.5934e+00],
          [-2.9289e-01,  1.5448e+00],
          [-3.6327e-01,  1.4954e+00],
          [-3.5503e-01,  1.4456e+00],
          [-2.7877e-01,  1.3956e+00],
          [-1.5258e-01,  1.3457e+00],
          [ 8.7423e-08,  1.2961e+00]])],
 [[20,
   21,
   22,
   

In [ ]:

op = apply_operator_set(model, full_prepared_operator)
if bconds==None:
    loss = torch.mean((op) ** 2)


true_b_val_list = []
b_val_list = []
b_pos_list = []

# we apply no  boundary conditions operators if they are all None

simpleform = False
for bcond in prepared_bconds:
    if bcond[1] == None:
        simpleform = True
    if bcond[1] != None:
        simpleform = False
        break
if simpleform:
    for bcond in prepared_bconds:
        b_pos_list.append(bcond[0])

        if len(bcond[2]) == bcond[2].shape[-1]:
            true_boundary_val = bcond[2].reshape(-1,1)
        else: 
            true_boundary_val = bcond[2]

        # true_boundary_val = bcond[2].reshape(-1, 1)
        true_b_val_list.append(true_boundary_val)
    true_b_val = torch.cat(true_b_val_list)
    b_op_val = model(prepared_grid)
    b_val = b_op_val[flatten_list(b_pos_list)]
else:
    for bcond in bconds:
        b_pos = bcond[0]
        b_pos_list.append(bcond[0])
        b_cond_operator = bcond[1]
        
        if len(bcond[2]) == bcond[2].shape[-1]:
            true_boundary_val = bcond[2].reshape(-1,1)
        else: 
            true_boundary_val = bcond[2]
        true_b_val_list.append(true_boundary_val)
        if b_cond_operator == None or b_cond_operator == [[1, [None], 1]]:
            b_op_val = model(prepared_grid)
        else:
            b_op_val = apply_operator_set(model, b_cond_operator)
        # take boundary values
        b_val_list.append(b_op_val[b_pos])
    true_b_val = torch.cat(true_b_val_list)
    b_val = torch.cat(b_val_list)

In [ ]:
len(true_b_val_list)

In [ ]:
point_sort_shift_loss_batch(model,prepared_grid,point_type,schrodinger_eq_real,bconds)

In [ ]:
op = op_dict_to_list(schrodinger_eq_real)
unified_operator = operator_unify(op)
unified_operator

In [ ]:
fin_diff_op = []
inner_order=1
boundary_order=2
nvars =list(grid_dict.values())[0].shape[-1]
h = 0.001
axes_scheme_type = 'central'
for term in unified_operator:
    fin_diff_list = []
    s_order_list = []
    const = term[0]
    vars_set = term[1]
    power = term[2]
    variables = term[3]
    for k, term in enumerate(vars_set):
        # None is for case where we need the fuction without derivatives
        if term != [None]:
            if axes_scheme_type == 'central':
                if inner_order==1:
                    scheme, direction_list = scheme_build(term, nvars, 'central')
                    s_order = sign_order(len(term), 'central', h=h)
                elif inner_order==2:
                    scheme, direction_list = second_order_scheme_build(term, nvars, 'central')
                    s_order = second_order_sign_order(len(term), 'central', h=h)
            else:
                if boundary_order == 1:
                    scheme, direction_list = scheme_build(term, nvars, axes_scheme_type)
                    s_order = sign_order(len(term), direction_list, h=h)
                elif boundary_order == 2:
                    scheme, direction_list = second_order_scheme_build(term, nvars, axes_scheme_type)
                    s_order = second_order_sign_order(len(term), direction_list, h=h)
        else:
            scheme = [None]
            s_order = [1]
        fin_diff_list.append(scheme)
        s_order_list.append(s_order)
    fin_diff_op.append([const, fin_diff_list, s_order_list, power,variables])

In [ ]:
for operator in full_prepared_operator:
    for term in operator:
        # it is may be int, function of grid or torch.Tensor
        coeff = term[0]
        # print(coeff)
        # this one contains shifted grids (see input_preprocessing module)
        shift_grid_list = term[1]
        print(len(shift_grid_list))
        # signs corresponding to a grid
        s_order_norm_list = term[2]
        # print(s_order_norm_list)
        # float that represents power of the differential term
        power = term[3]
        # print(power)
        # number of variables in equation
        variables = term[4]
        # initially it is an ones field
        der_term = torch.zeros_like(model(shift_grid_list[0][0])) + 1
        
        for j, scheme in enumerate(shift_grid_list):
            # every shift in grid we should add with correspoiding sign, so we start
            # from zeros
            grid_sum = torch.zeros_like(model(scheme[0])) #почему схема от 0?
            for k, grid in enumerate(scheme):
                # and add grid sequentially
                grid_sum += model(grid) * s_order_norm_list[j][k]
                # print(grid_sum)
                # print(grid_sum[:,0])
                # Here we want to apply differential operators for every term in the product
            der_term = der_term * grid_sum ** power[j]
        der_term = coeff * der_term


In [ ]:
der_term * grid_sum[:,0].reshape(-1,1) ** power[0]

In [ ]:
der_term